# Installing Dependencies

In [159]:
import sys

# Installing numpy
!{sys.executable} -m pip install numpy

# Installing pandas
!{sys.executable} -m pip install pandas

# Installing matplotlib
!{sys.executable} -m pip install matplotlib

# Installing statsmodels
!{sys.executable} -m pip install statsmodels

# Installing tqdm
!{sys.executable} -m pip install tqdm

# Installing requests
!{sys.executable} -m pip install requests

# Installing beautiful soup
!{sys.executable} -m pip install beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [160]:
# Import Statistic Libraries
import re
import requests

import numpy                    as np
import pandas                   as pd
import matplotlib.pyplot        as plt

# Import Utility Libraries
from typing                     import List, Dict
from tqdm                       import tqdm
from matplotlib.lines           import Line2D
from bs4                        import BeautifulSoup

# Utility functions

In [161]:
def get_page(number_of_page: int ) -> pd.DataFrame:
    page_url : str = f"https://kamernet.nl/en/for-rent/rooms-netherlands?radius=5&minSize=&maxRent=&searchview=1&typeAndCity=rooms-netherlands&pageNo={number_of_page}"

    page_response = requests.get(url= page_url)
    soup = BeautifulSoup(page_response.content, 'html.parser')
    listings = soup.find_all("a", { "class" : "MuiCard-root"})
    listing_dictionary : Dict[str : list] = {
        "street" : [],
        "city" : [],
        "size" : [],
        "furnishing" : [],
        "type" : [],
        "date" : [],
        "utilities": [],
        "price": [],
        "url": []
    }

    for listing in listings:
        locations = listing.findChildren("h6" , recursive=True)
        data_utilities = listing.findChildren("p", recursive=True)
        price = listing.findChild("h5", recursive = True)


        listing_street = locations[0].text
        listing_city = locations[1].text

        listing_size = 0
        listing_furnishing = ""
        listing_type = ""
        listing_date = ""
        listing_utilities = ""
        listing_price = 0

        if( len(data_utilities) == 5):
            listing_size = data_utilities[0].text
            listing_furnishing = data_utilities[1].text
            listing_type = data_utilities[2].text.replace(",", " ")
            listing_date = data_utilities[3].text
            listing_utilities = data_utilities[4].text
            listing_price = price.text

        if( len(data_utilities) ==  6):
            listing_size = data_utilities[0].text
            listing_furnishing = data_utilities[1].text
            listing_type = data_utilities[2].text.replace(",", " ") + data_utilities[3].text
            listing_date = data_utilities[4].text
            listing_utilities = data_utilities[5].text
            listing_price = price.text

        listing_dictionary['street'].append(listing_street)
        listing_dictionary['city'].append(listing_city)
        listing_dictionary['size'].append(listing_size)
        listing_dictionary['furnishing'].append(listing_furnishing)
        listing_dictionary['type'].append(listing_type)
        listing_dictionary['date'].append(listing_date)
        listing_dictionary['utilities'].append(listing_utilities[1:])
        listing_dictionary['price'].append(listing_price[1:])
        listing_dictionary['url'].append(f'https://kamernet.nl{listing.get('href')}')

    return pd.DataFrame(listing_dictionary)


In [162]:
def get_room_utilities(page_url: str) -> List[str]:
    result = []

    page_response = requests.get(url= page_url)
    soup = BeautifulSoup(page_response.content, 'html.parser')
    listing = soup.find("div", { "class" : "Details_gridContainer__nBfKx"})
    list_of_utilities = listing.findChildren('p', recursive=True)

    for utility in list_of_utilities:
        print(utility.text)
        result.append(utility.text)

# Scrapring


In [164]:
page_list : List[pd.DataFrame] = []

for i in tqdm(range(2, 42), desc="Processing"):

    page_list.append(get_page(i))

combined_df = pd.concat(page_list, ignore_index=True)

Processing: 100%|██████████| 40/40 [01:23<00:00,  2.08s/it]


In [170]:
utility_list = []

for list in tqdm(range(1, 10), desc="Processing"):
    row = combined_df.iloc[list]
    url = row[8]
    list_of_utilities_for_current_room = get_room_utilities(url)
    print(list_of_utilities_for_current_room)
    utility_list.append(list_of_utilities_for_current_room)

new_column = pd.DataFrame({
    "utilities": utility_list
})
combined_df

new_column

Processing:   0%|          | 0/9 [00:00<?, ?it/s]C:\Users\PC\AppData\Local\Temp\ipykernel_8652\2021424185.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = row[8]
Processing:  11%|█         | 1/9 [00:03<00:27,  3.48s/it]

None


Processing:  22%|██▏       | 2/9 [00:04<00:13,  1.99s/it]

None


Processing:  22%|██▏       | 2/9 [00:06<00:22,  3.21s/it]


KeyboardInterrupt: 

In [ ]:

output_file = "../data/combined_data.csv"
room_df.to_csv(output_file, index=False, sep=';')

'https://kamernet.nl/en/for-rent/room-zaandam/galjoenstraat/room-2207096'